# Create Scenarios

In [ ]:
from archetypal import UmiTemplateLibrary 
import pandas as pd
import numpy as np
import itertools
import os
import timeit
import random
from collections import Counter

In [ ]:
# Specify the input and output directories
template_directory = r"C:\Users\sb013698\Desktop\github\Deep Model Fusion for UBEM\Data Augmentation"
output_directory = r"C:\Users\sb013698\Desktop\github\Deep Model Fusion for UBEM\Data Augmentation\Scenarios"
save_directory = r"C:\Users\sb013698\Desktop\github\Deep Model Fusion for UBEM\Data Augmentation"

In [ ]:
# Insert the baseline template
template = UmiTemplateLibrary.open(template_directory)

In [ ]:
template_dict = {
    "Huzurevi": 0,
    "Ofis": 1,
    "Otel": 2,
    "Saglik Merkezi": 3,
    "Sosyal Bina": 4,
    "Spor Merkezi": 5,
}

# Select the sample archetype
archetype = "Huzurevi"
template_key = template_dict[archetype]
print(f"Template key for {archetype}: {template_key}")

### Creating Scenarios

In [ ]:
# Define parameter values
parameters = {
    "wall_u": [0.3, 0.6, 0.9, 1.2],
    "roof_u": [0.3, 0.7, 1.1],
    "window_u": [1.2, 1.8, 2.4],
    "heating_setpoint": [18, 20, 22, 24],
    "lpd": [2, 4, 6, 8, 10]
}

# Set the minimum number of scenarios each value must appear in
min_appearance = 4
num_random_scenarios = 5000
num_final_scenarios = 30

# Step 1: Generate a large pool of random scenarios
random_scenarios = [
    {param: random.choice(values) for param, values in parameters.items()}
    for _ in range(num_random_scenarios)
]

# Step 2: Count occurrences of each parameter value
def count_occurrences(scenarios, parameters):
    counts = {param: Counter() for param in parameters}
    for scenario in scenarios:
        for param, value in scenario.items():
            counts[param][value] += 1
    return counts

# Step 3: Filter the best 20 scenarios
def select_best_scenarios(scenarios, parameters, min_appearance, num_final_scenarios):
    selected_scenarios = []
    used_counts = {param: Counter() for param in parameters}
    
    for scenario in scenarios:
        # Check if adding this scenario helps meet min_appearance
        valid = True
        for param, value in scenario.items():
            if used_counts[param][value] >= min_appearance:
                valid = False
                break
        
        if valid or len(selected_scenarios) < num_final_scenarios:
            selected_scenarios.append(scenario)
            for param, value in scenario.items():
                used_counts[param][value] += 1

            # Stop if we have enough scenarios
            if len(selected_scenarios) == num_final_scenarios:
                break

    return selected_scenarios

# Count occurrences in the random pool
occurrences = count_occurrences(random_scenarios, parameters)

# Select the best scenarios
best_scenarios = select_best_scenarios(
    random_scenarios, parameters, min_appearance, num_final_scenarios
)

# Step 4: Create a DataFrame
scenarios_df = pd.DataFrame(best_scenarios)

# Add a "Scenario" column
scenarios_df['Scenario'] = [f"S{i+1}" for i in range(len(scenarios_df))]

# Reorder columns for readability
scenarios_df = scenarios_df[['Scenario'] + [col for col in scenarios_df.columns if col != 'Scenario']]

# Save to CSV if needed
scenarios_df.to_csv("random_scenarios.csv", index=False)

scenarios_df.head()

In [ ]:
# Create UMI templates using random scenarios
start = timeit.default_timer()

count = 1

# Assign parameter values to the template for each combination
for i in range(scenarios_df.shape[0]):
    
    # Make a copy of the template for each scenario
    new_template = template

    # Apply the same scenario values to each archetype within the template
    for archetype, template_key in template_dict.items():

        # Assign new values from scenarios
        new_template["ZoneLoads"][template_key]["LightingPowerDensity"] = float(scenarios_df.loc[i, "lpd"])
        new_template["ZoneConditionings"][template_key]["HeatingSetpoint"] = float(scenarios_df.loc[i, "heating_setpoint"])
        new_template["ZoneConstructionSets"][template_key]["Facade"]["r_value"] = float(1/scenarios_df.loc[i, "wall_u"])
        new_template["ZoneConstructionSets"][template_key]["Roof"]["r_value"] = float(1/scenarios_df.loc[i, "roof_u"])

        # Adjust window thickness based on scenario values
        original_thickness = new_template.WindowConstructions[template_key].Layers[0].Thickness
        original_r = new_template.WindowConstructions[template_key].r_value
        sceanario_r = 1 / scenarios_df.loc[i, "window_u"]
        window_ratio = sceanario_r / original_r
        new_thickness = original_thickness * window_ratio
        new_template.WindowConstructions[template_key].Layers[0].Thickness = new_thickness

    # Save a single JSON file for each scenario
    file_name = f"S{count}.json"
    new_template.to_json(os.path.join(output_directory, file_name), indent=4)
    
    # Increment the count for unique filenames
    count += 1

# Stop the timer
stop = timeit.default_timer()
run_time = stop - start
print(f"Total run time (sec): {run_time:.1f}")

In [ ]:
# Update U-Values from UMI templates per scenario
wall_list = list()
roof_list = list()
window_list = list()

n_scenarios = 30

for i in range(1, n_scenarios+1):
    
    template_dir = os.path.join(output_directory, f"S{i}.json")
    template = UmiTemplateLibrary.open(template_dir)
    template_key = 0
    wall_u = template["ZoneConstructionSets"][template_key]["Facade"]["u_value"]
    roof_u = template["ZoneConstructionSets"][template_key]["Roof"]["u_value"]
    window_u = template.WindowConstructions[template_key].u_value
    wall_list.append(round(wall_u, 3))
    roof_list.append(round(roof_u, 3))
    window_list.append(round(window_u, 3))

updated_scenarios = scenarios_df[["Scenario", "lpd", "heating_setpoint"]].copy()
updated_scenarios["wall_u"] = wall_list
updated_scenarios["roof_u"] = roof_list
updated_scenarios["window_u"] = window_list
updated_scenarios

# Save updated scenarios to a CSV file
file_name = os.path.join(save_directory, "static_scenarios.csv")
updated_scenarios.to_csv(file_name, index=False)

# END